# NODE ANALYSIS PROGRAM

```
Python code to generate nodal equations from a circuit net list
by Tony
Date: April 17, 2017
Name: node analysis.ipynb
Synopsis: This program will read in a spice type file and compute the node equations.
Description: Follows Erik Cheever's Analysis of  Resistive Circuits [page](http://www.swarthmore.edu/NatSci/echeeve1/Ref/mna/MNA1.html) to generate modified nodal equations.  I somewhat followed his matlab file.  
Requires: Python version 3 or higher
Author: Tony Cirineo
Revision History
7/1/2015: Ver 1 - coding started, derived from network.c code
8/18/2017
change approach, now implementing a modified nodal analysis
8/19/2017
Wrote some code to generate symbolic matrices, work ok, so heading down the sympy path. Basic debugging finished, but still need to verify some circuits using Ls and Cs.
Still to do:
Clean up comments
Add controlled sources
Add Op Amps
Add coupled inductors



```

In [1]:
import os
from sympy import *
import numpy as np
import pandas as pd
init_printing()

In [2]:
# initialize some variables
num_passives = 0    # number of passive elements
num_v_ind = 0    # number of independent voltage sources
num_i_ind = 0    # number of independent current sources

num_opamps = 0   # Number of op amps
num_vcvs = 0     # number of sources
num_vccs = 0
num_cccs = 0
num_ccvs = 0
num_cpld_ind = 0

#### open file and preprocess it
- remove blank lines and comments
- converts all lower case to upper
- removes extra spaces between entries
- count number of entries on each line, make sure the count is correct

In [3]:
fn = 'example2'
fd1 = open(fn+'.net','r')
content = fd1.readlines()
content = [x.strip() for x in content]  #remove leading and trailing white space
# remove empty lines
while '' in content:
    content.pop(content.index(''))

# remove comment lines, these start with a asterisk *
content = [n for n in content if not n.startswith('*')]
# converts all lower case to upper
content = [x.upper() for x in content]
# removes extra spaces between entries
content = [' '.join(x.split()) for x in content]

In [4]:
branch_cnt = len(content)
# chech number of entries on each line
for i in range(branch_cnt):
    x = content[i][0]
    tk_cnt = len(content[i].split())

    if (x == 'R') or (x == 'L') or (x == 'C'):
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_passives += 1
    elif x == 'V':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_v_ind += 1
    elif x == 'I':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("2had {:d} items and should only be 4".format(tk_cnt))
        num_i_ind += 1
    elif x == 'O':
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("2had {:d} items and should only be 4".format(tk_cnt))
        num_opamps += 1
    elif x == 'E':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vcvs += 1
    elif x == 'G':
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 6".format(tk_cnt))
        num_vccs += 1
    elif x == 'F':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_cccs += 1
    elif x == 'H':
        if (tk_cnt != 5):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 5".format(tk_cnt))
        num_ccvs += 1
    elif x == 'K':
        if (tk_cnt != 4):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("had {:d} items and should only be 4".format(tk_cnt))
        num_cpld_ind += 1
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

#### PARSER
- puts branch elements into structure
- counts number of nodes

In [5]:
# try puting the branch structre in a pandas data frame  <<-- needs some clean up eg not using phase
count = []
element = []      # type of element
p_node = []       # positive node
n_node = []       # neg node
cp_node = []      # controlling pos node of branch
cn_node = []      # controlling neg node of branch
source_type = []  # 1 = AC, 2 = DC source, 3 for multi-terminal
value = []        # value of element or voltage
phase = []        # AC phase
source_imp = []   # source impedance

df = pd.DataFrame(index=count, columns=['element','p node','n node','cp node','cn node',
    'source type','value','phase','source imp'])

#### Functions to load branch elements into data frame

In [6]:
# loads AC and DC voltage and current sources into branch structure
def indep_source(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'source type'] = tk[3]
    if tk[3] == 'AC':
        df.loc[br_nu,'value'] = float(tk[4])
        df.loc[br_nu,'phase'] = float(tk[5])
        df.loc[br_nu,'source imp'] = float(tk[6])
    if tk[3] == 'DC':
        df.loc[br_nu,'value'] = float(tk[4])
        df.loc[br_nu,'source imp'] = float(tk[6])

In [7]:
# loads passive elements into branch structure
def pass_element(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'value'] = float(tk[3])

In [8]:
'''
loads multi-terminal sub-networks
into branch structure
Types:
E - VCVS
G - VCCS
F - CCCS
H - CCVS
not implemented yet:
K - Coupled inductors
O - Op Amps
'''
def sub_network(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'cp node'] = int(tk[3])
    df.loc[br_nu,'cn node'] = int(tk[4])
    df.loc[br_nu,'value'] = float(tk[5])

In [9]:
# scan df and get largest node number
def count_nodes():
    # need to ckeck that nodes are consecutive
    # fill array with node numbers
    p = np.zeros(branch_cnt+1)
    for i in range(branch_cnt-1):
        p[df['p node'][i]] = df['p node'][i]
        p[df['n node'][i]] = df['n node'][i]

    # find the largest node number
    if df['n node'].max() > df['p node'].max():
        largest = df['n node'].max()
    else:
        largest =  df['p node'].max()

        largest = int(largest)
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if p[i] == 0:
            print("nodes not in continuous order");

    return largest

In [10]:
# load branches into data frame
for i in range(branch_cnt):
    x = content[i][0]

    if (x == 'R') or (x == 'L') or (x == 'C'):
        pass_element(i)
    elif (x == 'V') or (x == 'I'):
        indep_source(i)
    elif (x == 'E') or (x == 'F') or (x == 'G') or (x == 'H'):
        sub_network(i)
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

# count number of nodes
num_nodes = count_nodes()

In [11]:
# print a report
print('number of branches: {:d}'.format(branch_cnt))
print('number of nodes: {:d}'.format(num_nodes))
print('number of passive components: {:d}'.format(num_passives))
print('number of independent voltage sources: {:d}'.format(num_v_ind))
print('number of independent current sources: {:d}'.format(num_i_ind))

# not implemented yet
print('number of op amps: {:d}'.format(num_opamps))
print('number of E - VCVS: {:d}'.format(num_vcvs))
print('number of G - VCCS: {:d}'.format(num_vccs))
print('number of F - CCCS: {:d}'.format(num_cccs))
print('number of F - CCCS: {:d}'.format(num_ccvs))
print('number of K - Coupled inductors: {:d}'.format(num_cpld_ind))

number of branches: 5
number of nodes: 2
number of passive components: 3
number of independent voltage sources: 1
number of independent current sources: 1
number of op amps: 0
number of E - VCVS: 0
number of G - VCCS: 0
number of F - CCCS: 0
number of F - CCCS: 0
number of K - Coupled inductors: 0


In [12]:
# store the data frame as a pickle file
df.to_pickle(fn+'.pkl')

In [13]:
# initialize some symbolic matrix with zeros
# A is formed by [[G, C] [B, D]]
# Z = [I,E]
# X = [V, J]
V = zeros(num_nodes,1)
I = zeros(num_nodes,1)
G = zeros(num_nodes,num_nodes)
s = Symbol('s')

if (num_v_ind+num_opamps) != 0:
    B = zeros(num_nodes,num_v_ind+num_opamps)
    C = zeros(num_v_ind+num_opamps,num_nodes)
    D = zeros(num_v_ind+num_opamps,num_v_ind+num_opamps)
    E = zeros(num_v_ind+num_opamps,1)
    J = zeros(num_v_ind+num_opamps,1)

In [14]:
'''
G matrix
the G matrix is nxn and is determined by the interconnections between the passive circuit elements (RLC's)
The G matrix is an nxn matrix formed in two steps
1) Each element in the diagonal matrix is equal to the sum of the conductance (one over the resistance) of each element connected to the corresponding node.  So the first diagonal element is the sum of conductances connected to node 1, the second diagonal element is the sum of conductances connected to node 2, and so on.
2) The off diagonal elements are the negative conductance of the element connected to the pair of corresponding node.  Therefore a resistor between nodes 1 and 2 goes into the G matrix at location (1,2) and locations (2,1).
'''
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'R':
        g = 1/sympify(df.loc[i,'element'])
    if x == 'L':
        g = 1/(s/sympify(df.loc[i,'element']))
    if x == 'C':
        g = sympify(df.loc[i,'element'])*s

    if (x == 'R') or (x == 'L') or (x == 'C'):
        # If neither side of the element is connected to ground
        # then subtract it from appropriate location in matrix.
        if (n1 != 0) and (n2 != 0):
            G[n1-1,n2-1] += -g
            G[n2-1,n1-1] += -g

        # If node 1 is connected to graound, add element to diagonal
        # of matrix.
        if n1 != 0:
            G[n1-1,n1-1] += g

        # Ditto for node 2.
        if n2 != 0:
            G[n2-1,n2-1] += g

In [15]:
# The I matrix is an nx1 matrix with each element of the matrix corresponding to a particular node.  The value of each element of i is determined by the sum of current sources into the corresponding node.  If there are no current sources connected to the node, the value is zero.
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the passive elements, save conductance to temp value
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'I':
        g = sympify(df.loc[i,'element'])
        # sum the current into each node
        if n1 != 0:
            I[n1-1] += g
        if n2 != 0:
            I[n2-1] -= g

In [16]:
# The V matrix is an nx1 matrix formed of the node voltages.  Each element in v corresponds to the voltage at the equivalent node in the circuit
for i in range(num_nodes):
    V[i] = sympify('v{:d}'.format(i+1))

In [17]:
'''
B Matrix
Rules for making the B matrix
The B matrix is an nxm matrix with only 0, 1 and -1 elements.  Each location in the matrix corresponds to a particular voltage source (first dimension) or a node (second dimension).  If the positive terminal of the ith voltage source is connected to node k, then the element (i,k) in the B matrix is a 1.  If the negative terminal of the ith voltage source is connected to node k, then the element (i,k) in the B matrix is a -1.  Otherwise, elements of the B matrix are zero.
'''
# First handle the case of the independent voltage sources.
sn = 0   # count source number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the independent voltage sources
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if num_v_ind+num_opamps > 1:
            if n1 != 0:
                B[n1-1,sn] = 1
            if n2 != 0:
                B[n2-1,sn] = -1
            sn += 1   #increment source count
        else:
            if n1 != 0:
                B[n1-1] = 1
            if n2 != 0:
                B[n2-1] = -1
    # Op amps not implemented

In [18]:
# The J matrix is an mx1 matrix, with one entry for the current through each voltage source.
sn = 0   # count source number
for i in range(branch_cnt):
    # process all the passive elements
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        J[sn] = sympify('I_{:s}'.format(df.loc[i,'element']))
        sn += 1

In [19]:
'''
The C matrix is an mxn matrix with only 0, 1 and -1 elements.  Each location in the matrix corresponds to a particular node (first dimension) or voltage source (second dimension).  If the positive terminal of the ith voltage source is connected to node k, then the element (k,i) in the C matrix is a 1.  If the negative terminal of the ith voltage source is connected to node k, then the element (k,i) in the C matrix is a -1.  Otherwise, elements of the C matrix are zero.
'''
sn = 0   # count source number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the independent voltage sources
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if num_v_ind+num_opamps > 1:
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
            sn += 1   #increment source count
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1


In [20]:
# The D matrix is an mxm matrix that is composed entirely of zeros.  (It can be non-zero if dependent sources are considered.)
D

In [21]:
# the E matrix is mx1 and holds the values of the independent voltage sources.
sn = 0   # count source number
for i in range(branch_cnt):
    # process all the passive elements
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        E[sn] = sympify(df.loc[i,'element'])
        sn += 1

In [22]:
'''
Form the Z matrix
The z matrix holds the independent voltage and current sources and is the combination of 2 smaller matrices i and e.
The z matrix is (m+n)x1, n is the number of nodes, and m is the number of independent voltage sources
'''
Z = I[:] + E[:]

In [23]:
'''
Form the X matrix
The I matrix is nx1 and contains the sum of the currents through the passive elements into the corresponding node (either zero, or the sum of independent current sources).
The E matrix is mx1 and holds the values of the independent voltage sources.
'''
X = V[:] + J[:]

In [24]:
'''
Form the A matrix
The A matrix is (m+n)x(m+n) and will be developed as the combination of 4 smaller matrices, G, B, C, and D.
'''
n = num_nodes
m = num_v_ind
A = zeros(m+n,m+n)
for i in range(n):
    for j in range(n):
        A[i,j] = G[i,j]

if num_v_ind+num_opamps > 1:
    for i in range(n):
        for j in range(m):
            A[i,n+j] = B[i,j]
            A[n+j,i] = C[j,i]
else:
    for i in range(n):
        A[i,n] = B[i]
        A[n,i] = C[i]

In [25]:
sn = 0   # count source number
for i in range(branch_cnt):
    n1 = df.loc[i,'p node']
    n2 = df.loc[i,'n node']
    # process all the independent voltage sources
    x = df.loc[i,'element'][0]   #get 1st letter of element name
    if x == 'V':
        if num_v_ind+num_opamps > 1:
            if n1 != 0:
                C[sn,n1-1] = 1
            if n2 != 0:
                C[sn,n2-1] = -1
            sn += 1   #increment source count
        else:
            if n1 != 0:
                C[n1-1] = 1
            if n2 != 0:
                C[n2-1] = -1


In [26]:
A

In [27]:
Z

In [28]:
X

In [29]:
# generate the node equations
n = num_nodes
m = num_v_ind
eq1 = 0
equ = zeros(m+n,1)
for i in range(n+m):
    for j in range(n+m):
        eq1 += A[j,i]*X[j]
    equ[i] = Eq(eq1,Z[i])
    eq1 = 0

In [30]:
equ

In [31]:
R1, R2, R3 = symbols('R1 R2 R3')
v1, v2, v3 = symbols('v1 v2 v3')
VB, IS, IVB= symbols('VB IS IVB')

In [32]:
equ1a = equ.subs({R1:5})
equ1a = equ1a.subs({R2:3})
equ1a = equ1a.subs({R3:10})

equ1a = equ1a.subs({VB:30})
equ1a = equ1a.subs({IS:2})

In [33]:
equ1a

In [37]:
equ1a.row_del(0)

In [38]:
equ1a

In [39]:
solve(equ1a,[v1,v2])